# Jogoteca (Flask)

Este notebook replica o arquivo `app.py`, organizando os comentários como explicações em Markdown e mantendo o código funcional em células Python.

A estrutura de pastas esperada pelo Flask é:
- `templates/` para os arquivos HTML (`login.html`, `lista.html`, `novo.html`, etc.)
- `static/` para arquivos estáticos como `bootstrap.css`

Nos templates, carregue o CSS com Jinja:
```html
<link rel="stylesheet" href="{{ url_for('static', filename='bootstrap.css') }}">
```

## Inicialização do Flask e Configuração da Chave Secreta

- Obrigatório para iniciar o Flask: criamos a aplicação com `Flask(__name__)`.
- Sem `secret_key`, a `session` (cookie) não funciona; por isso definimos `app.secret_key`.

In [17]:
from flask import Flask, render_template, request, redirect, session, flash, url_for

# criar a aplicação Flask
app = Flask(__name__)

# sem o secret key, o session(cookie), nao funciona
app.secret_key = 'Rondi'

## Modelo de Usuário e Usuários de Exemplo

- `Usuario` representa um usuário do sistema (nome, nickname, senha).
- Criamos 3 usuários de exemplo e montamos um dicionário `usuarios` onde a chave é o `nickname` e o valor é o objeto `Usuario`.
- Esse dicionário funciona como um “mini banco de dados” para autenticação.

In [ ]:
class Usuario:
    def __init__(self, nome, nickname, senha):
        self.nome = nome
        self.nickname = nickname
        self.senha = senha
        
usuario1 = Usuario("Bruno Divino", "BD", "alohomora")
usuario2 = Usuario("Camila Ferreira", "Mila", "paozinho")
usuario3 = Usuario("Guilherme Louro", "Cake", "python_eh_vida")
usuario4 = Usuario('Rondinelle Oliveira','Rondi', '123')

usuarios = { 
    usuario1.nickname: usuario1, 
    usuario2.nickname: usuario2,
    usuario3.nickname: usuario3,
    usuario4.nickname: usuario4
}

## Modelo de Jogo e Lista Global de Jogos

- `Jogo` representa um jogo com nome, categoria e console.
- Criamos 3 jogos de exemplo e mantemos uma lista global `lista` (fora das rotas) para que todas as rotas possam acessá-la.

In [19]:
class Jogo:
    def __init__(self, nome, categoria, console):
        self.nome = nome
        self.categoria = categoria
        self.console = console
 
# instanciado jogos padrao ja que nao tem banco de dados
jogo1 = Jogo("The Legend of Zelda: Breath of the Wild", "Aventura", "Nintendo Switch")
jogo2 = Jogo("God of War Ragnarök", "Ação", "PlayStation 5")
jogo3 = Jogo("Halo Infinite", "FPS", "Xbox Series X")  

# lista com os jogos da classe, tem que estar do lado de fora para ser global
lista = [jogo1, jogo2, jogo3]

## Rotas de Interface: Login, Lista e Novo Jogo

- `/login` sempre renderiza `login.html` (formulário).
- `/` mostra a lista de jogos, mas primeiro verifica se há usuário logado na sessão.
- `/novo` mostra o formulário de novo jogo e também exige usuário logado.

In [20]:
# pagina de login
@app.route('/login')
def login():
    # sempre direciona para a pagina login.html
    return render_template('login.html')

# pagina com a tabela de jogos, interface vem da tabela lista.html
@app.route('/')
def index():
    # SE cookie vazio, pagina nao entra | TEM que estar logado
    if 'usuario_logado' not in session or session['usuario_logado'] == None:
        # volta para a pagina de login
        return redirect(url_for('login'))   
    # cookie com nome de usuario == True | direciona para pagina com a tabela de jogos
    return render_template('lista.html', jogos = lista, titulo = 'Jogos')

# pagina para criar os jogos, interface vem da tabela novo.html
@app.route('/novo', methods = ['GET'])
def novo_jogo():
    # SE cookie vazio, pagina nao entra | TEM que estar logado
    if 'usuario_logado' not in session or session['usuario_logado'] == None:
        # volta para a pagina de login
        return redirect(url_for('login'))
    # cookie com nome de usuario == True | direciona para pagina de criar jogos
    return render_template('novo.html', titulo = 'Inserir Novo Jogo')

## Rota de Criação (POST) e PRG

- `/criar` recebe os dados do formulário de `novo.html` via método POST.
- Coleta `nome`, `categoria`, `console`, cria um objeto `Jogo` e adiciona à `lista`.
- Aplica o padrão POST-Redirect-GET (PRG): após o POST, redireciona para `/` (GET) para evitar reenvio de formulário ao atualizar a página.

In [21]:
# pagina de redirecionamento | recebe dados da pagina de criar jogos | Apenas uma ponte que leva para a pagina com a tabela
# dados vem do formulario da pagina novo.html
@app.route('/criar', methods = ['POST'])
def criar_jogo():
    # vai no html e pega os dados do input
    nome = request.form['nome']
    categoria = request.form['categoria']
    console = request.form['console']
    # instancia a classe baseado nos inputs recebidos do HTML
    novo_jogo = Jogo(nome, categoria, console)
    # inclui os dados do input na lista
    lista.append(novo_jogo)
    # direciona para a pagina da tabela ja com o novo jogo adicionado
    return redirect(url_for('index'))

## Autenticação de Usuário

- `/autenticar` recebe `usuario` (nickname) e `senha` do formulário `login.html`.
- Busca no dicionário `usuarios` pelo nickname e compara a senha.
- Se der certo, grava `session['usuario_logado']` e usa `flash()` para mensagens.
- Em caso de falha, exibe mensagem e redireciona para o login.

In [ ]:
# pagina de validacao de login | dados vem do formulario da pagina login.html
@app.route('/autenticar', methods=['POST'])
def autenticar():
    # SE o nickname digitado estiver na lista que tem como chave o nickname
    if request.form['usuario'] in usuarios:
        # usuario vira uma instancia do objeto | Ex lista_usuarios['nickname'].nome --> acessa o valor que e um objeto
        usuario = usuarios[request.form['usuario']]
        if request.form['senha'] == usuario.senha:
            session['usuario_logado'] = usuario.nickname
            flash(usuario.nickname + ' logado com sucesso')
        # direciona para a pagina de criar jogos
        return redirect(url_for('novo_jogo'))
    else:
        flash('Usuario ou senha incorreto.')
        # SE a senha estiver errada, volta eternamente para a pagina de login
        return redirect(url_for('index'))

## Logout

- Limpa a sessão (`session['usuario_logado'] = None`).
- Mostra uma mensagem com `flash()` e redireciona para `/login`.

In [23]:
# pagina de logout, desativa os cookies
@app.route('/logout')
def logout():
    # apaga os cookies da funcao session
    session['usuario_logado'] = None
    flash('Logout efetuado com sucesso!')
    # volta para a pagina de login
    return redirect(url_for('login'))

## Executando o App (apenas em desenvolvimento)

- A célula abaixo inicia o servidor Flask no modo debug.
- Observação: ao rodar dentro do notebook, o kernel ficará ocupado pelo servidor.
- Alternativa: iniciar pelo terminal PowerShell (recomendado) no diretório do projeto.

No PowerShell (opcional):
```powershell
$env:FLASK_APP="app.py"; $env:FLASK_ENV="development"; python app.py
```

In [24]:
# iniciar o servidor (opcional rodar via notebook)
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.3.82:8080
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)

 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.3.82:8080
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\rondi\AppData\Roaming\Python\Python313\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
